In [1]:
# pulls from the FantasyPros and ESPN API

## **Important: The dataframes are built using csv files in the current working directory so 
# do not delete or comment out these functions or lines of code that create the csv files

In [2]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display
from datetime import datetime
import nfl_data_py as nfl
import os

In [3]:
# Define the template URLs for the QB, RB, and WR positions
espn_urls = {
    "QB": "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page={page}&limit=50&category=offense%3Apassing&sort=passing.passingYards%3Adesc&season={year}&seasontype={seasontype}",
    "RB": "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page={page}&limit=50&category=offense%3Arushing&sort=rushing.rushingYards%3Adesc&season={year}&seasontype={seasontype}",
    "WR": "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page={page}&limit=50&category=offense%3Areceiving&sort=receiving.receivingYards%3Adesc&season={year}&seasontype={seasontype}"
}

In [4]:
# **IMPORTANT: this function outputs the combined qb-betting lines df
# Get current year and week for NFL
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4) ## *** Reset the date at the start of the NFL season ***
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Set the current NFL year and week
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [5]:
# Adjust the fetch function to return both data and pagination info for verification
def fetch_position_data_with_verification(position, url_template):
    page = 1
    all_players = []
    total_pages = 1  # Default to 1 page unless pagination indicates more
    
    while True:
        # Construct the API URL for the current page
        url = url_template.format(page=page, year=current_year, seasontype=seasontype)
        response = requests.get(url)
        
        # If response is successful, process the data
        if response.status_code == 200:
            data = response.json()
            athletes = data.get('athletes', [])
            
            # Get pagination information for verification
            if page == 1:
                pagination = data.get('pagination', {})
                total_pages = pagination.get('pages', 1)  # Total number of pages
            
            if not athletes:
                break  # Stop if no more athletes are available
            
            for athlete_data in athletes:
                athlete = athlete_data['athlete']

                # Extract relevant data for the base columns (excluding stats)
                player_info = {
                    'year': current_year,
                    'week': current_week,
                    'player_id': athlete.get('id', 'N/A'),
                    'player': athlete.get('displayName', 'N/A'),
                    'position': position,
                    'team': athlete.get('teamShortName', 'N/A')
                }

                all_players.append(player_info)
            
            page += 1  # Increment to the next page
        else:
            break  # Stop if there's an error in fetching data
    
    return all_players, total_pages

In [6]:
# Function to convert fetched data into a DataFrame
def create_dataframe(position_data):
    return pd.DataFrame(position_data)
    
# Updated process function to include verification step
def process_and_verify_position_data():
    # Fetch data for each position and track total pages
    qb_data, qb_pages = fetch_position_data_with_verification("QB", espn_urls["QB"])
    rb_data, rb_pages = fetch_position_data_with_verification("RB", espn_urls["RB"])
    wr_data, wr_pages = fetch_position_data_with_verification("WR", espn_urls["WR"])
    
    # Convert fetched data into DataFrames
    df_qb = create_dataframe(qb_data)
    df_rb = create_dataframe(rb_data)
    df_wr = create_dataframe(wr_data)
    
    # Verification output
    print(f"QB: Fetched {len(df_qb)} rows across {qb_pages} pages.")
    print(f"RB: Fetched {len(df_rb)} rows across {rb_pages} pages.")
    print(f"WR: Fetched {len(df_wr)} rows across {wr_pages} pages.")
    
    # Display the first few rows for review
    display(df_qb.head())
    display(df_rb.head())
    display(df_wr.head())
    
    return df_qb, df_rb, df_wr

# Call the function to fetch, verify, and display the data
df_qb, df_rb, df_wr = process_and_verify_position_data()

QB: Fetched 63 rows across 2 pages.
RB: Fetched 217 rows across 5 pages.
WR: Fetched 369 rows across 8 pages.


,year,week,player_id,player,position,team
0,2024,6,15864,Geno Smith,QB,SEA
1,2024,6,4361741,Brock Purdy,QB,SF
2,2024,6,2577417,Dak Prescott,QB,DAL
3,2024,6,4432577,C.J. Stroud,QB,HOU
4,2024,6,14880,Kirk Cousins,QB,ATL


,year,week,player_id,player,position,team
0,2024,6,4360569,Jordan Mason,RB,SF
1,2024,6,3043078,Derrick Henry,RB,BAL
2,2024,6,3929630,Saquon Barkley,RB,PHI
3,2024,6,4047365,Josh Jacobs,RB,GB
4,2024,6,4241416,Chuba Hubbard,RB,CAR


,year,week,player_id,player,position,team
0,2024,6,4258173,Nico Collins,WR,HOU
1,2024,6,4362628,Ja'Marr Chase,WR,CIN
2,2024,6,4047650,DK Metcalf,WR,SEA
3,2024,6,4262921,Justin Jefferson,WR,MIN
4,2024,6,4362249,Jayden Reed,WR,GB


In [7]:
# Function to generate FantasyPros URLs based on the positions
# Function to generate FantasyPros URLs based on the positions
def generate_fantasy_pros_urls(season, positions=None, week=None, scoring=None):
    base_url = f"https://api.fantasypros.com/public/v2/json/nfl/{season}/projections"
    # If positions is not provided, default to QB, RB, WR. Otherwise, use the list directly.
    positions_list = ['QB', 'RB', 'WR'] if positions is None else positions  # Remove split
    scoring_str = scoring.replace("'", "") if scoring else None
    generated_urls = []

    for position in positions_list:
        params = {'position': position}
        if season:
            params['season'] = season
        if week:
            params['week'] = week
        if scoring:
            params['scoring'] = scoring_str
        query_string = requests.compat.urlencode(params)
        full_url = f"{base_url}?{query_string}"
        generated_urls.append(full_url)

    return generated_urls

# Function to fetch data from FantasyPros API
def fetch_data(url, headers=None):
    response = requests.get(url, headers=headers)
    try:
        response.raise_for_status()
        return response.json()  # Return the JSON data
    except requests.RequestException as e:
        print(f"Failed to retrieve {url}. Error: {e}")
        return None

# Function to fetch and handle FantasyPros data for given positions and stats
def fetch_fantasy_pros_data(season, positions=None, week=None, scoring=None):
    api_key = os.getenv('api_key')
    if not api_key:
        print("API key is not set.")
        return None
    
    headers = {'x-api-key': api_key}
    urls = generate_fantasy_pros_urls(season, positions, week, scoring)
    all_data = []
    
    for url in urls:
        print(f"Fetching FantasyPros data from: {url}")
        response = fetch_data(url, headers)
        if response and 'players' in response:
            players_data = response['players']
            for player in players_data:
                # Extract general columns
                player_info = {
                    'name': player['name'],
                    'points': player['stats'].get('points', 0),
                    'points_ppr': player['stats'].get('points_ppr', 0),
                    'points_half': player['stats'].get('points_half', 0)
                }
                # Extract position-specific columns based on position
                position = player.get('position_id')
                if position == 'QB':
                    player_info.update({
                        'passing_attempts': player['stats'].get('pass_att', 0),
                        'passing_completions': player['stats'].get('pass_cmp', 0),
                        'passing_yards': player['stats'].get('pass_yds', 0),
                        'passing_tds': player['stats'].get('pass_tds', 0)
                    })
                elif position == 'RB':
                    player_info.update({
                        'rushing_attempts': player['stats'].get('rush_att', 0),
                        'rushing_yards': player['stats'].get('rush_yds', 0),
                        'rushing_tds': player['stats'].get('rush_tds', 0),
                        'receptions': player['stats'].get('rec_rec', 0),
                        'reception_yards': player['stats'].get('rec_yds', 0),
                        'reception_tds': player['stats'].get('rec_tds', 0)
                    })
                elif position == 'WR':
                    player_info.update({
                        'receptions': player['stats'].get('rec_rec', 0),
                        'reception_yards': player['stats'].get('rec_yds', 0),
                        'reception_tds': player['stats'].get('rec_tds', 0)
                    })
                all_data.append(player_info)
    
    return pd.DataFrame(all_data)

In [8]:
# Function to merge ESPN data with FantasyPros data, keeping only the relevant columns for each position
def merge_espn_fantasypros(espn_df, fantasypros_df, position):
    # Extract relevant columns based on position
    if position == 'QB':
        # Extract only QB relevant columns
        fantasypros_df = fantasypros_df[['name', 'points', 'points_ppr', 'points_half', 
                                         'passing_attempts', 'passing_completions', 'passing_yards', 'passing_tds']]
    elif position == 'RB':
        # Extract only RB relevant columns
        fantasypros_df = fantasypros_df[['name', 'points', 'points_ppr', 'points_half', 
                                         'rushing_attempts', 'rushing_yards', 'rushing_tds', 
                                         'receptions', 'reception_yards', 'reception_tds']]
    elif position == 'WR':
        # Extract only WR relevant columns
        fantasypros_df = fantasypros_df[['name', 'points', 'points_ppr', 'points_half', 
                                         'receptions', 'reception_yards', 'reception_tds']]

    # Merge on 'player' from ESPN and 'name' from FantasyPros
    merged_df = pd.merge(espn_df, fantasypros_df, left_on='player', right_on='name', how='left')
    
    # Drop the redundant 'name' column from FantasyPros
    merged_df.drop(columns=['name'], inplace=True)
    
    return merged_df

In [9]:
# Function to fetch, merge, and save ESPN and FantasyPros data for all positions
def process_and_merge_fantasypros_data(df_qb, df_rb, df_wr, scoring='STD'):
    # Fetch current season and week dynamically
    current_week = get_current_week()
    season = datetime.now().year

    # Fetch FantasyPros data for all positions
    fantasypros_data = fetch_fantasy_pros_data(season=season, positions=['QB', 'RB', 'WR'], week=current_week, scoring=scoring)

    # Merging ESPN dataframes with FantasyPros data, keeping only relevant columns
    df_qb_merged = merge_espn_fantasypros(df_qb, fantasypros_data, 'QB')
    df_rb_merged = merge_espn_fantasypros(df_rb, fantasypros_data, 'RB')
    df_wr_merged = merge_espn_fantasypros(df_wr, fantasypros_data, 'WR')

    # Define output CSV file names
    qb_output_csv = 'qb_projections_fp.csv'
    rb_output_csv = 'rb_projections_fp.csv'
    wr_output_csv = 'wr_projections_fp.csv'

    # Save the DataFrames to CSV files
    df_qb_merged.to_csv(qb_output_csv, index=False)
    print(f"Data successfully written to {qb_output_csv}")
    
    df_rb_merged.to_csv(rb_output_csv, index=False)
    print(f"Data successfully written to {rb_output_csv}")
    
    df_wr_merged.to_csv(wr_output_csv, index=False)
    print(f"Data successfully written to {wr_output_csv}")

    # Display the merged dataframes
    display(df_qb_merged.head())
    display(df_rb_merged.head())
    display(df_wr_merged.head())
    
    return df_qb_merged, df_rb_merged, df_wr_merged

# Call the function to fetch, merge, save, and display the data
df_qb_merged, df_rb_merged, df_wr_merged = process_and_merge_fantasypros_data(df_qb, df_rb, df_wr)


Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=QB&season=2024&week=6&scoring=STD
Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=RB&season=2024&week=6&scoring=STD
Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=WR&season=2024&week=6&scoring=STD
Data successfully written to qb_projections_fp.csv
Data successfully written to rb_projections_fp.csv
Data successfully written to wr_projections_fp.csv


,year,week,player_id,player,position,team,points,points_ppr,points_half,passing_attempts,passing_completions,passing_yards,passing_tds
0,2024,6,15864,Geno Smith,QB,SEA,17.74,17.74,17.74,37.66,25.00,269.30,1.43
1,2024,6,4361741,Brock Purdy,QB,SF,18.36,18.36,18.36,33.34,22.47,273.36,1.66
2,2024,6,2577417,Dak Prescott,QB,DAL,18.41,18.41,18.41,36.38,24.26,271.11,1.78
3,2024,6,4432577,C.J. Stroud,QB,HOU,17.58,17.58,17.58,33.45,21.87,258.16,1.61
4,2024,6,14880,Kirk Cousins,QB,ATL,17.12,17.12,17.12,34.58,22.74,259.82,1.85


,year,week,player_id,player,position,team,points,points_ppr,points_half,rushing_attempts,rushing_yards,rushing_tds,receptions,reception_yards,reception_tds
0,2024,6,4360569,Jordan Mason,RB,SF,13.85,15.47,14.66,18.94,86.23,0.66,1.61,11.70,0.05
1,2024,6,3043078,Derrick Henry,RB,BAL,15.55,16.92,16.23,17.97,87.29,0.91,1.37,11.60,0.06
2,2024,6,3929630,Saquon Barkley,RB,PHI,15.06,17.93,16.50,19.56,85.88,0.61,2.87,21.00,0.15
3,2024,6,4047365,Josh Jacobs,RB,GB,11.69,13.90,12.80,16.19,68.85,0.45,2.21,16.82,0.10
4,2024,6,4241416,Chuba Hubbard,RB,CAR,12.41,15.30,13.85,16.61,72.28,0.44,2.89,21.38,0.09


,year,week,player_id,player,position,team,points,points_ppr,points_half,receptions,reception_yards,reception_tds
0,2024,6,4258173,Nico Collins,WR,HOU,NaN,NaN,NaN,NaN,NaN,NaN
1,2024,6,4362628,Ja'Marr Chase,WR,CIN,11.75,17.85,14.80,6.10,83.23,0.59
2,2024,6,4047650,DK Metcalf,WR,SEA,9.96,15.15,12.55,5.19,74.00,0.45
3,2024,6,4262921,Justin Jefferson,WR,MIN,0.00,0.00,0.00,0.00,0.00,0.00
4,2024,6,4362249,Jayden Reed,WR,GB,10.27,15.36,12.82,5.10,65.44,0.42


In [10]:
## **IMPORTANT *** DO NOT DELETE OR COMMENT OUT - THE CSV output is needed
def scrape_salary_changes():
    # URL of the FantasyPros salary changes page
    url = "https://www.fantasypros.com/daily-fantasy/nfl/fanduel-salary-changes.php"
    
    # Fetch the page content
    response = requests.get(url)
    
    # Check if the page was fetched successfully
    if response.status_code != 200:
        print(f"Failed to fetch the page. Status code: {response.status_code}")
        return
    
    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Locate the table containing the salary changes (assuming it's the first table)
    table = soup.find('table')  # Adjust if necessary based on the page structure
    
    # Extract the table headers
    headers = [header.text for header in table.find_all('th')]
    
    # Extract the table rows
    rows = []
    for row in table.find_all('tr')[1:]:  # Skip the header row
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]  # Clean up whitespace
        rows.append(cols)
    
    # Create a DataFrame with the scraped data
    salary_changes_df = pd.DataFrame(rows, columns=headers)
    
    # Save to a CSV file
    salary_changes_df.to_csv('salary_changes.csv', index=False)
    print("Salary changes data saved to 'salary_changes.csv'")
    
    # Display the first few rows of the DataFrame
    display(salary_changes_df.head())
    
    return salary_changes_df

# Run the function to scrape and display salary changes
scrape_salary_changes()

Salary changes data saved to 'salary_changes.csv'


,ECR,Player,Kickoff,Opp,This Week,Last Week,Difference
0,-,Riley Sharp (BAL - TE),Sun 1:00PM,WAS,"$4,000","$4,000",0
1,225,John Metchie III (HOU - WR),Sun 1:00PM,@NE,"$4,300","$4,200",+100
2,-,Snoop Conner (DAL - RB),Sun 4:25PM,DET,"$4,000","$4,000",0
3,347,Kevin Harris (NE - RB),Sun 1:00PM,HOU,"$4,000","$4,000",0
4,3,Jayden Daniels (WAS - QB),Sun 1:00PM,@BAL,"$8,700","$8,700",0


,ECR,Player,Kickoff,Opp,This Week,Last Week,Difference
0,-,Riley Sharp (BAL - TE),Sun 1:00PM,WAS,"$4,000","$4,000",0
1,225,John Metchie III (HOU - WR),Sun 1:00PM,@NE,"$4,300","$4,200",+100
2,-,Snoop Conner (DAL - RB),Sun 4:25PM,DET,"$4,000","$4,000",0
3,347,Kevin Harris (NE - RB),Sun 1:00PM,HOU,"$4,000","$4,000",0
4,3,Jayden Daniels (WAS - QB),Sun 1:00PM,@BAL,"$8,700","$8,700",0
...,...,...,...,...,...,...,...
887,-,Xavier Johnson (HOU - WR),Sun 1:00PM,@NE,"$4,000","$4,000",0
888,349,Ryan Miller (TB - TE),Sun 1:00PM,@NO,"$4,000","$4,000",0
889,-,D.J. Montgomery (IND - WR),Sun 1:00PM,@TEN,"$4,000","$4,000",0
890,-,Owen Wright (BAL - RB),Sun 1:00PM,WAS,"$4,000","$4,000",0


In [11]:
# **IMPORTANT *** DO NOT DELETE OR COMMENT OUT - THE CSV output is needed
# Function to check for existing CSV and run scrape_salary_changes() if file is not found
def check_and_read_salary_changes():
    file_name = 'salary_changes.csv'
    
    # Check if the file exists
    if not os.path.exists(file_name):
        print(f"{file_name} not found. Running scrape_salary_changes()...")
        scrape_salary_changes()
    
    # Read the CSV file
    salary_changes_df = pd.read_csv(file_name)
    return salary_changes_df

# Updated function to process and split the salary changes data by position
def process_and_save_salary_changes_by_position():
    # Step 1: Check and load the data
    salary_changes_df = check_and_read_salary_changes()
    
    # Step 2: Split 'Player' column into three separate columns (FirstName, LastName, Team-Position)
    salary_changes_df[['FirstName', 'LastName', 'Team-Position']] = salary_changes_df['Player'].str.extract(r'(\w+)\s+(\w+)\s+\((.*)\)')
    
    # Step 3: Drop the original 'Player' column and the first column (Rankings)
    salary_changes_df.drop(columns=['Player', salary_changes_df.columns[0]], inplace=True)
    
    # Step 4: Split 'Team-Position' into separate 'Team' and 'Position' columns
    salary_changes_df[['Team', 'Position']] = salary_changes_df['Team-Position'].str.extract(r'(\w+)\s*-\s*(\w+)')
    
    # Step 5: Drop 'Team-Position' and any unwanted columns like 'Kickoff' and 'Opp'
    salary_changes_df.drop(columns=['Team-Position', 'Kickoff', 'Opp'], inplace=True, errors='ignore')
    
    # Step 6: Reorder the columns to match the desired order
    salary_changes_df = salary_changes_df[['FirstName', 'LastName', 'Team', 'Position', 'This Week', 'Last Week', 'Difference']]
    
    # Step 7: Split the DataFrame by position
    df_qb = salary_changes_df[salary_changes_df['Position'] == 'QB']
    df_wr = salary_changes_df[salary_changes_df['Position'] == 'WR']
    df_rb = salary_changes_df[salary_changes_df['Position'] == 'RB']
    
    # Step 8: Save each position-specific DataFrame to a CSV file
    df_qb.to_csv('qb_salary_changes.csv', index=False)
    df_wr.to_csv('wr_salary_changes.csv', index=False)
    df_rb.to_csv('rb_salary_changes.csv', index=False)
    
    print("CSV files saved for QB, WR, and RB salary changes.")
    
    # Display the first few rows of each DataFrame for verification
    display(df_qb.head())
    display(df_wr.head())
    display(df_rb.head())
    
    return df_qb, df_wr, df_rb

# Run the function to process and save the salary changes by position
df_qb_salary, df_wr_salary, df_rb_salary = process_and_save_salary_changes_by_position()


CSV files saved for QB, WR, and RB salary changes.


,FirstName,LastName,Team,Position,This Week,Last Week,Difference
4,Jayden,Daniels,WAS,QB,"$8,700","$8,700",0
12,Kedon,Slovis,HOU,QB,"$6,000","$6,000",0
15,Bo,Nix,DEN,QB,"$6,700","$6,600",+100
21,Max,Duggan,LAC,QB,"$6,000",-,-
39,Easton,Stick,LAC,QB,"$6,000",-,-


,FirstName,LastName,Team,Position,This Week,Last Week,Difference
1,Metchie,III,HOU,WR,"$4,300","$4,200",+100
5,Ricky,Pearsall,SF,WR,"$4,000","$4,000",0
6,Joseph,Ngata,PHI,WR,"$4,000",-,-
7,Anthony,Gould,IND,WR,"$4,000","$4,000",0
9,Trey,Palmer,TB,WR,"$4,200","$4,400",-200


,FirstName,LastName,Team,Position,This Week,Last Week,Difference
2,Snoop,Conner,DAL,RB,"$4,000","$4,000",0
3,Kevin,Harris,NE,RB,"$4,000","$4,000",0
11,Eric,Gray,NYG,RB,"$4,700","$4,200",+500
16,Austin,Jones,WAS,RB,"$4,000","$4,000",0
18,Nathaniel,Peat,DAL,RB,"$4,000","$4,000",0


In [12]:
## **IMPORTANT *** DO NOT DELETE OR COMMENT OUT - THE CSV output is needed
# Ensure salary files exist
def check_and_read_csv_files():
    # List of projection and salary files
    projection_files = ['qb_projections_fp.csv', 'rb_projections_fp.csv', 'wr_projections_fp.csv']
    salary_files = ['qb_salary_changes.csv', 'rb_salary_changes.csv', 'wr_salary_changes.csv']
    
    # Check for missing projection files
    projections_missing = False
    for file in projection_files:
        if not os.path.exists(file):
            print(f"Projections file {file} not found in {os.getcwd()}. Please generate the file.")
            projections_missing = True
    
    # Check for missing salary files
    salary_missing = False
    for file in salary_files:
        if not os.path.exists(file):
            print(f"Salary file {file} not found in {os.getcwd()}. Please generate the file.")
            salary_missing = True
    
    if projections_missing or salary_missing:
        return None, None, None, None, None, None
    
    # Read and return projection files
    df_qb_projections = pd.read_csv('qb_projections_fp.csv')
    df_rb_projections = pd.read_csv('rb_projections_fp.csv')
    df_wr_projections = pd.read_csv('wr_projections_fp.csv')
    
    # Read and return salary files
    df_qb_salary = pd.read_csv('qb_salary_changes.csv')
    df_rb_salary = pd.read_csv('rb_salary_changes.csv')
    df_wr_salary = pd.read_csv('wr_salary_changes.csv')
    
    return df_qb_projections, df_rb_projections, df_wr_projections, df_qb_salary, df_rb_salary, df_wr_salary

# Merge the salary data with the projections based on firstName and lastName
def merge_salary_and_projections(projections_df, salary_df):
    # Ensure salary_df has FirstName and LastName already
    if 'FirstName' not in salary_df.columns or 'LastName' not in salary_df.columns:
        print("Error: 'FirstName' and/or 'LastName' columns are missing in salary DataFrame.")
        return None
    
    # Ensure projections have player names split into firstName and lastName
    projections_df[['firstName', 'lastName']] = projections_df['player'].str.split(' ', n=1, expand=True)
    
    # Merge the two dataframes
    merged_df = pd.merge(projections_df, salary_df, left_on=['firstName', 'lastName'], right_on=['FirstName', 'LastName'], how='left')
    
    # Drop redundant columns from salary data
    merged_df.drop(columns=['FirstName', 'LastName'], inplace=True)
    
    return merged_df

# Final process to save the merged data
def process_and_save_final_data():
    # Read the CSV files
    df_qb_projections, df_rb_projections, df_wr_projections, df_qb_salary, df_rb_salary, df_wr_salary = check_and_read_csv_files()
    
    if df_qb_projections is None or df_rb_projections is None or df_wr_projections is None:
        print("Missing projections or salary DataFrames. Exiting process.")
        return

    # Merge the projections and salary changes
    qb_projections_salary_merged_df = merge_salary_and_projections(df_qb_projections, df_qb_salary)
    rb_projections_salary_merged_df = merge_salary_and_projections(df_rb_projections, df_rb_salary)
    wr_projections_salary_merged_df = merge_salary_and_projections(df_wr_projections, df_wr_salary)
    
    # Save the final merged data to CSV files
    if qb_projections_salary_merged_df is not None:
        qb_projections_salary_merged_df.to_csv('qb_projections_salary_merged.csv', index=False)
        print("qb_projections_salary_merged.csv saved successfully.")
    
    if rb_projections_salary_merged_df is not None:
        rb_projections_salary_merged_df.to_csv('rb_projections_salary_merged.csv', index=False)
        print("rb_projections_salary_merged.csv saved successfully.")
    
    if wr_projections_salary_merged_df is not None:
        wr_projections_salary_merged_df.to_csv('wr_projections_salary_merged.csv', index=False)
        print("wr_projections_salary_merged.csv saved successfully.")

    # Display the first few rows of each merged DataFrame for verification
    if qb_projections_salary_merged_df is not None:
        display(qb_projections_salary_merged_df.head())
    if rb_projections_salary_merged_df is not None:
        display(rb_projections_salary_merged_df.head())
    if wr_projections_salary_merged_df is not None:
        display(wr_projections_salary_merged_df.head())

# Run the final process
process_and_save_final_data()

qb_projections_salary_merged.csv saved successfully.
rb_projections_salary_merged.csv saved successfully.
wr_projections_salary_merged.csv saved successfully.


,year,week,player_id,player,position,team,points,points_ppr,points_half,passing_attempts,passing_completions,passing_yards,passing_tds,firstName,lastName,Team,Position,This Week,Last Week,Difference
0,2024,6,15864,Geno Smith,QB,SEA,17.74,17.74,17.74,37.66,25.00,269.30,1.43,Geno,Smith,SEA,QB,"$7,900","$7,400",+500
1,2024,6,4361741,Brock Purdy,QB,SF,18.36,18.36,18.36,33.34,22.47,273.36,1.66,Brock,Purdy,SF,QB,"$7,800","$7,700",+100
2,2024,6,2577417,Dak Prescott,QB,DAL,18.41,18.41,18.41,36.38,24.26,271.11,1.78,Dak,Prescott,DAL,QB,"$8,000","$8,000",0
3,2024,6,4432577,C.J. Stroud,QB,HOU,17.58,17.58,17.58,33.45,21.87,258.16,1.61,C.J.,Stroud,NaN,NaN,NaN,NaN,NaN
4,2024,6,14880,Kirk Cousins,QB,ATL,17.12,17.12,17.12,34.58,22.74,259.82,1.85,Kirk,Cousins,ATL,QB,"$7,400","$7,000",+400


,year,week,player_id,player,position,team,points,points_ppr,points_half,rushing_attempts,...,receptions,reception_yards,reception_tds,firstName,lastName,Team,Position,This Week,Last Week,Difference
0,2024,6,4360569,Jordan Mason,RB,SF,13.85,15.47,14.66,18.94,...,1.61,11.70,0.05,Jordan,Mason,SF,RB,"$8,900","$8,700",+200
1,2024,6,3043078,Derrick Henry,RB,BAL,15.55,16.92,16.23,17.97,...,1.37,11.60,0.06,Derrick,Henry,BAL,RB,"$9,100","$9,200",-100
2,2024,6,3929630,Saquon Barkley,RB,PHI,15.06,17.93,16.50,19.56,...,2.87,21.00,0.15,Saquon,Barkley,PHI,RB,"$9,200",-,-
3,2024,6,4047365,Josh Jacobs,RB,GB,11.69,13.90,12.80,16.19,...,2.21,16.82,0.10,Josh,Jacobs,GB,RB,"$7,600","$7,000",+600
4,2024,6,4241416,Chuba Hubbard,RB,CAR,12.41,15.30,13.85,16.61,...,2.89,21.38,0.09,Chuba,Hubbard,CAR,RB,"$7,400","$6,800",+600


,year,week,player_id,player,position,team,points,points_ppr,points_half,receptions,reception_yards,reception_tds,firstName,lastName,Team,Position,This Week,Last Week,Difference
0,2024,6,4258173,Nico Collins,WR,HOU,NaN,NaN,NaN,NaN,NaN,NaN,Nico,Collins,HOU,WR,"$8,900","$8,800",+100
1,2024,6,4362628,Ja'Marr Chase,WR,CIN,11.75,17.85,14.80,6.10,83.23,0.59,Ja'Marr,Chase,NaN,NaN,NaN,NaN,NaN
2,2024,6,4047650,DK Metcalf,WR,SEA,9.96,15.15,12.55,5.19,74.00,0.45,DK,Metcalf,SEA,WR,"$8,300","$8,200",+100
3,2024,6,4262921,Justin Jefferson,WR,MIN,0.00,0.00,0.00,0.00,0.00,0.00,Justin,Jefferson,NaN,NaN,NaN,NaN,NaN
4,2024,6,4362249,Jayden Reed,WR,GB,10.27,15.36,12.82,5.10,65.44,0.42,Jayden,Reed,GB,WR,"$7,700","$7,200",+500


In [13]:
## **IMPORTANT *** DO NOT DELETE OR COMMENT OUT - THE CSV output is needed
# Ensure salary files exist
def check_and_read_csv_files():
    # List of projection and salary files
    projection_files = ['qb_projections_fp.csv', 'rb_projections_fp.csv', 'wr_projections_fp.csv']
    salary_files = ['qb_salary_changes.csv', 'rb_salary_changes.csv', 'wr_salary_changes.csv']
    
    # Check for missing projection files
    projections_missing = False
    for file in projection_files:
        if not os.path.exists(file):
            print(f"Projections file {file} not found in {os.getcwd()}. Please generate the file.")
            projections_missing = True
    
    # Check for missing salary files
    salary_missing = False
    for file in salary_files:
        if not os.path.exists(file):
            print(f"Salary file {file} not found in {os.getcwd()}. Please generate the file.")
            salary_missing = True
    
    if projections_missing or salary_missing:
        return None, None, None, None, None, None
    
    # Read and return projection files
    df_qb_projections = pd.read_csv('qb_projections_fp.csv')
    df_rb_projections = pd.read_csv('rb_projections_fp.csv')
    df_wr_projections = pd.read_csv('wr_projections_fp.csv')
    
    # Read and return salary files
    df_qb_salary = pd.read_csv('qb_salary_changes.csv')
    df_rb_salary = pd.read_csv('rb_salary_changes.csv')
    df_wr_salary = pd.read_csv('wr_salary_changes.csv')
    
    return df_qb_projections, df_rb_projections, df_wr_projections, df_qb_salary, df_rb_salary, df_wr_salary

# Merge the salary data with the projections based on firstName and lastName
def merge_salary_and_projections(projections_df, salary_df):
    # Ensure salary_df has FirstName and LastName already
    if 'FirstName' not in salary_df.columns or 'LastName' not in salary_df.columns:
        print("Error: 'FirstName' and/or 'LastName' columns are missing in salary DataFrame.")
        return None
    
    # Ensure projections have player names split into firstName and lastName
    projections_df[['firstName', 'lastName']] = projections_df['player'].str.split(' ', n=1, expand=True)
    
    # Merge the two dataframes
    merged_df = pd.merge(projections_df, salary_df, left_on=['firstName', 'lastName'], right_on=['FirstName', 'LastName'], how='left')
    
    # Drop redundant columns from salary data
    merged_df.drop(columns=['FirstName', 'LastName', 'Team', 'Position'], inplace=True)
    
    # Reorder columns to move 'firstName' before 'lastName' right after 'player'
    cols = list(merged_df.columns)
    player_index = cols.index('player')
    first_last_name = ['firstName', 'lastName']
    
    # Insert 'firstName' before 'lastName'
    for name in first_last_name[::-1]:  # Reverse the order to insert correctly
        cols.insert(player_index + 1, cols.pop(cols.index(name)))
    
    merged_df = merged_df[cols]
    
    return merged_df

# Final process to save the merged data
def process_and_save_final_data():
    # Read the CSV files
    df_qb_projections, df_rb_projections, df_wr_projections, df_qb_salary, df_rb_salary, df_wr_salary = check_and_read_csv_files()
    
    if df_qb_projections is None or df_rb_projections is None or df_wr_projections is None:
        print("Missing projections or salary DataFrames. Exiting process.")
        return

    # Merge the projections and salary changes
    qb_projections_salary_merged_df = merge_salary_and_projections(df_qb_projections, df_qb_salary)
    rb_projections_salary_merged_df = merge_salary_and_projections(df_rb_projections, df_rb_salary)
    wr_projections_salary_merged_df = merge_salary_and_projections(df_wr_projections, df_wr_salary)
    
    # Save the final merged data to CSV files
    if qb_projections_salary_merged_df is not None:
        qb_projections_salary_merged_df.to_csv('qb_projections_salary_merged.csv', index=False)
        print("qb_projections_salary_merged.csv saved successfully.")
    
    if rb_projections_salary_merged_df is not None:
        rb_projections_salary_merged_df.to_csv('rb_projections_salary_merged.csv', index=False)
        print("rb_projections_salary_merged.csv saved successfully.")
    
    if wr_projections_salary_merged_df is not None:
        wr_projections_salary_merged_df.to_csv('wr_projections_salary_merged.csv', index=False)
        print("wr_projections_salary_merged.csv saved successfully.")

    # Display the first few rows of each merged DataFrame for verification
    if qb_projections_salary_merged_df is not None:
        display(qb_projections_salary_merged_df.head())
    if rb_projections_salary_merged_df is not None:
        display(rb_projections_salary_merged_df.head())
    if wr_projections_salary_merged_df is not None:
        display(wr_projections_salary_merged_df.head())

# Run the final process
process_and_save_final_data()


qb_projections_salary_merged.csv saved successfully.
rb_projections_salary_merged.csv saved successfully.
wr_projections_salary_merged.csv saved successfully.


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,points_half,passing_attempts,passing_completions,passing_yards,passing_tds,This Week,Last Week,Difference
0,2024,6,15864,Geno Smith,Geno,Smith,QB,SEA,17.74,17.74,17.74,37.66,25.00,269.30,1.43,"$7,900","$7,400",+500
1,2024,6,4361741,Brock Purdy,Brock,Purdy,QB,SF,18.36,18.36,18.36,33.34,22.47,273.36,1.66,"$7,800","$7,700",+100
2,2024,6,2577417,Dak Prescott,Dak,Prescott,QB,DAL,18.41,18.41,18.41,36.38,24.26,271.11,1.78,"$8,000","$8,000",0
3,2024,6,4432577,C.J. Stroud,C.J.,Stroud,QB,HOU,17.58,17.58,17.58,33.45,21.87,258.16,1.61,NaN,NaN,NaN
4,2024,6,14880,Kirk Cousins,Kirk,Cousins,QB,ATL,17.12,17.12,17.12,34.58,22.74,259.82,1.85,"$7,400","$7,000",+400


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,points_half,rushing_attempts,rushing_yards,rushing_tds,receptions,reception_yards,reception_tds,This Week,Last Week,Difference
0,2024,6,4360569,Jordan Mason,Jordan,Mason,RB,SF,13.85,15.47,14.66,18.94,86.23,0.66,1.61,11.70,0.05,"$8,900","$8,700",+200
1,2024,6,3043078,Derrick Henry,Derrick,Henry,RB,BAL,15.55,16.92,16.23,17.97,87.29,0.91,1.37,11.60,0.06,"$9,100","$9,200",-100
2,2024,6,3929630,Saquon Barkley,Saquon,Barkley,RB,PHI,15.06,17.93,16.50,19.56,85.88,0.61,2.87,21.00,0.15,"$9,200",-,-
3,2024,6,4047365,Josh Jacobs,Josh,Jacobs,RB,GB,11.69,13.90,12.80,16.19,68.85,0.45,2.21,16.82,0.10,"$7,600","$7,000",+600
4,2024,6,4241416,Chuba Hubbard,Chuba,Hubbard,RB,CAR,12.41,15.30,13.85,16.61,72.28,0.44,2.89,21.38,0.09,"$7,400","$6,800",+600


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,points_half,receptions,reception_yards,reception_tds,This Week,Last Week,Difference
0,2024,6,4258173,Nico Collins,Nico,Collins,WR,HOU,NaN,NaN,NaN,NaN,NaN,NaN,"$8,900","$8,800",+100
1,2024,6,4362628,Ja'Marr Chase,Ja'Marr,Chase,WR,CIN,11.75,17.85,14.80,6.10,83.23,0.59,NaN,NaN,NaN
2,2024,6,4047650,DK Metcalf,DK,Metcalf,WR,SEA,9.96,15.15,12.55,5.19,74.00,0.45,"$8,300","$8,200",+100
3,2024,6,4262921,Justin Jefferson,Justin,Jefferson,WR,MIN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
4,2024,6,4362249,Jayden Reed,Jayden,Reed,WR,GB,10.27,15.36,12.82,5.10,65.44,0.42,"$7,700","$7,200",+500


In [14]:
## **IMPORTANT *** DO NOT DELETE OR COMMENT OUT - THE CSV output is needed
def scrape_rostered_percentage_over_weeks(positions, scoring='PPR', range_type='week'):
    current_week = get_current_week()
    
    for position in positions:
        # Initialize an empty DataFrame to accumulate data
        rostered_df = pd.DataFrame()
        
        # Loop over weeks from current_week down to 1
        for week in range(current_week, 0, -1):
            # Construct the URL for the given position and week
            url = f"https://www.fantasypros.com/nfl/stats/{position}.php?week={week}&scoring={scoring}&range={range_type}"
            
            # Fetch the page content
            response = requests.get(url)
            
            # Check if the page was fetched successfully
            if response.status_code != 200:
                print(f"Failed to fetch the page for {position} week {week}. Status code: {response.status_code}")
                continue
            
            # Parse the page content using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Locate the table containing the stats data
            table = soup.find('table')
            
            # Extract the table headers
            headers = [header.text.strip() for header in table.find_all('th')]
            
            # Find the indices of the "Player" and "ROST" columns
            if "Player" not in headers or "ROST" not in headers:
                print(f"'Player' or 'ROST' column not found for {position} week {week}.")
                continue
            
            player_index = headers.index("Player")
            rost_index = headers.index("ROST")
            
            # Extract the table rows
            rows = []
            for row in table.find_all('tr')[1:]:  # Skip the header row
                cols = row.find_all('td')
                cols = [ele.get_text(strip=True) for ele in cols]  # Clean up whitespace
                
                # Only process rows with the expected number of columns
                if len(cols) >= rost_index + 1:
                    player_name = cols[player_index]  # Player name
                    rost_percentage = cols[rost_index]  # Rostered percentage
                    rows.append([player_name, rost_percentage])
            
            # Create a DataFrame for the current week
            week_df = pd.DataFrame(rows, columns=["Player", f"wk{week}"])
            
            # Convert the rostered percentage to float (remove '%' sign)
            week_df[f"wk{week}"] = week_df[f"wk{week}"].str.replace('%', '').astype(float)
            
            if rostered_df.empty:
                # First week's data, initialize the cumulative DataFrame
                rostered_df = week_df
            else:
                # Merge with the cumulative DataFrame
                rostered_df = pd.merge(rostered_df, week_df, on='Player', how='outer')
        
        # After looping through all weeks, sort columns in descending order
        # First, get the list of columns except 'Player'
        cols = rostered_df.columns.tolist()
        cols.remove('Player')
        # Sort the week columns in descending order
        cols_sorted = sorted(cols, key=lambda x: int(x[2:]), reverse=True)
        # Rearrange the columns
        rostered_df = rostered_df[['Player'] + cols_sorted]
        
        # Optionally, you can sort the DataFrame by current week's ROST
        if f"wk{current_week}" in rostered_df.columns:
            rostered_df = rostered_df.sort_values(by=f"wk{current_week}", ascending=False)
        
        # Save to a CSV file specific to the position
        csv_filename = f'rostered_percentage_{position}_weeks_1_to_{current_week}.csv'
        rostered_df.to_csv(csv_filename, index=False)
        print(f"Rostered percentage data for {position} saved to '{csv_filename}'")
        
        # Display the first few rows of the DataFrame
        display(rostered_df.head())
    
    print("Scraping completed for all positions.")

# Example usage: Scrape rostered percentage over weeks for all positions
positions = ['qb', 'wr', 'rb', 'te', 'flex', 'dst']
scrape_rostered_percentage_over_weeks(positions, scoring='PPR', range_type='week')


Rostered percentage data for qb saved to 'rostered_percentage_qb_weeks_1_to_6.csv'


,Player,wk6,wk5,wk4,wk3,wk2,wk1
62,Josh Allen(BUF),100.0,100.0,100.0,100.0,100.0,100.0
73,Lamar Jackson(BAL),100.0,100.0,100.0,100.0,100.0,100.0
47,Jalen Hurts(PHI),99.9,99.9,99.9,99.9,99.9,99.9
87,Patrick Mahomes II(KC),98.2,98.2,98.2,98.2,98.2,98.2
16,C.J. Stroud(HOU),97.9,97.9,97.9,97.9,97.9,97.9


Rostered percentage data for wr saved to 'rostered_percentage_wr_weeks_1_to_6.csv'


,Player,wk6,wk5,wk4,wk3,wk2,wk1
169,Justin Jefferson(MIN),100.0,100.0,100.0,100.0,100.0,100.0
11,Amon-Ra St. Brown(DET),100.0,100.0,100.0,100.0,100.0,100.0
41,CeeDee Lamb(DAL),100.0,100.0,100.0,100.0,100.0,100.0
114,Ja'Marr Chase(CIN),100.0,100.0,100.0,100.0,100.0,100.0
0,A.J. Brown(PHI),99.9,99.9,99.9,99.9,99.9,99.9


Rostered percentage data for rb saved to 'rostered_percentage_rb_weeks_1_to_6.csv'


...

Rostered percentage data for flex saved to 'rostered_percentage_flex_weeks_1_to_6.csv'


,Player,wk6,wk5,wk4,wk3,wk2,wk1
62,Josh Allen(BUF),100.0,100.0,100.0,100.0,100.0,100.0
73,Lamar Jackson(BAL),100.0,100.0,100.0,100.0,100.0,100.0
47,Jalen Hurts(PHI),99.9,99.9,99.9,99.9,99.9,99.9
87,Patrick Mahomes II(KC),98.2,98.2,98.2,98.2,98.2,98.2
16,C.J. Stroud(HOU),97.9,97.9,97.9,97.9,97.9,97.9


Rostered percentage data for dst saved to 'rostered_percentage_dst_weeks_1_to_6.csv'


,Player,wk6,wk5,wk4,wk3,wk2,wk1
27,San Francisco 49ers(SF),94.8,94.8,94.8,94.8,94.8,94.8
26,Pittsburgh Steelers(PIT),93.2,93.2,93.2,93.2,93.2,93.2
24,New York Jets(NYJ),83.1,83.1,83.1,83.1,83.1,83.1
25,Philadelphia Eagles(PHI),81.5,81.5,81.5,81.5,81.5,81.5
9,Denver Broncos(DEN),79.4,79.4,79.4,79.4,79.4,79.4


Scraping completed for all positions.


In [15]:
# Function to check and read all necessary CSV files (renamed)
def check_and_read_proj_salary_rostered_csv_files():
    # Projection + salary merged files
    qb_proj_salary_file = 'qb_projections_salary_merged.csv'
    rb_proj_salary_file = 'rb_projections_salary_merged.csv'
    wr_proj_salary_file = 'wr_projections_salary_merged.csv'
    
    # Rostered percentage files
    qb_rostered_file = 'rostered_percentage_qb_weeks_1_to_6.csv'
    rb_rostered_file = 'rostered_percentage_rb_weeks_1_to_6.csv'
    wr_rostered_file = 'rostered_percentage_wr_weeks_1_to_6.csv'

    # Check and read projections + salary merged CSVs
    if not os.path.exists(qb_proj_salary_file) or not os.path.exists(rb_proj_salary_file) or not os.path.exists(wr_proj_salary_file):
        print("Merged projection and salary files missing. Please generate the files.")
        return None, None, None

    df_qb_proj_salary = pd.read_csv(qb_proj_salary_file)
    df_rb_proj_salary = pd.read_csv(rb_proj_salary_file)
    df_wr_proj_salary = pd.read_csv(wr_proj_salary_file)

    # Check and read rostered percentage CSVs
    if not os.path.exists(qb_rostered_file) or not os.path.exists(rb_rostered_file) or not os.path.exists(wr_rostered_file):
        print("Rostered percentage files missing. Please generate the files.")
        return None, None, None

    df_qb_rostered = pd.read_csv(qb_rostered_file)
    df_rb_rostered = pd.read_csv(rb_rostered_file)
    df_wr_rostered = pd.read_csv(wr_rostered_file)

    return df_qb_proj_salary, df_rb_proj_salary, df_wr_proj_salary, df_qb_rostered, df_rb_rostered, df_wr_rostered

# Merge the rostered data with the projections + salary data
def merge_with_rostered_data(projections_df, rostered_df):
    # Split the 'Player' column in the rostered data into 'firstName' and 'lastName'
    rostered_df[['firstName', 'lastName']] = rostered_df['Player'].str.extract(r'(\w+)\s+(.+)\(')
    
    # Merge based on firstName and lastName
    merged_df = pd.merge(projections_df, rostered_df[['firstName', 'lastName', 'wk6', 'wk5', 'wk4', 'wk3', 'wk2', 'wk1']], 
                         on=['firstName', 'lastName'], how='left')

    # Rename the columns as per your request
    column_renames = {
        'This Week': 'This Week Salary',
        'Last Week': 'Last Week Salary',
        'Difference': 'Salary Differential',
        'wk6': 'wk6 %rostered',
        'wk5': 'wk5 %rostered',
        'wk4': 'wk4 %rostered',
        'wk3': 'wk3 %rostered',
        'wk2': 'wk2 %rostered',
        'wk1': 'wk1 %rostered'
    }
    
    merged_df.rename(columns=column_renames, inplace=True)
    
    return merged_df

# Final function to process everything and save the final CSVs
def process_and_save_final_rostered_data():
    # Read the CSV files using the renamed function
    df_qb_proj_salary, df_rb_proj_salary, df_wr_proj_salary, df_qb_rostered, df_rb_rostered, df_wr_rostered = check_and_read_proj_salary_rostered_csv_files()
    
    if df_qb_proj_salary is None or df_rb_proj_salary is None or df_wr_proj_salary is None:
        print("Missing DataFrames. Exiting process.")
        return

    # Merge with rostered percentage data and rename the final DataFrames
    qb_projections_salary_rostered_merged_df = merge_with_rostered_data(df_qb_proj_salary, df_qb_rostered)
    rb_projections_salary_rostered_merged_df = merge_with_rostered_data(df_rb_proj_salary, df_rb_rostered)
    wr_projections_salary_rostered_merged_df = merge_with_rostered_data(df_wr_proj_salary, df_wr_rostered)

    # Save the final merged DataFrames to CSV files
    qb_projections_salary_rostered_merged_df.to_csv('qb_projections_salary_rostered_merged.csv', index=False)
    print("qb_projections_salary_rostered_merged.csv saved successfully.")
    
    rb_projections_salary_rostered_merged_df.to_csv('rb_projections_salary_rostered_merged.csv', index=False)
    print("rb_projections_salary_rostered_merged.csv saved successfully.")
    
    wr_projections_salary_rostered_merged_df.to_csv('wr_projections_salary_rostered_merged.csv', index=False)
    print("wr_projections_salary_rostered_merged.csv saved successfully.")

    # Display the first few rows of the final DataFrames for verification
    display(qb_projections_salary_rostered_merged_df.head())
    display(rb_projections_salary_rostered_merged_df.head())
    display(wr_projections_salary_rostered_merged_df.head())

# Run the process
process_and_save_final_rostered_data()


qb_projections_salary_rostered_merged.csv saved successfully.
rb_projections_salary_rostered_merged.csv saved successfully.
wr_projections_salary_rostered_merged.csv saved successfully.


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,...,passing_tds,This Week Salary,Last Week Salary,Salary Differential,wk6 %rostered,wk5 %rostered,wk4 %rostered,wk3 %rostered,wk2 %rostered,wk1 %rostered
0,2024,6,15864,Geno Smith,Geno,Smith,QB,SEA,17.74,17.74,...,1.43,"$7,900","$7,400",+500,56.8,56.8,56.8,56.8,56.8,56.8
1,2024,6,4361741,Brock Purdy,Brock,Purdy,QB,SF,18.36,18.36,...,1.66,"$7,800","$7,700",+100,91.3,91.3,91.3,91.3,91.3,91.3
2,2024,6,2577417,Dak Prescott,Dak,Prescott,QB,DAL,18.41,18.41,...,1.78,"$8,000","$8,000",0,94.5,94.5,94.5,94.5,94.5,94.5
3,2024,6,4432577,C.J. Stroud,C.J.,Stroud,QB,HOU,17.58,17.58,...,1.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024,6,14880,Kirk Cousins,Kirk,Cousins,QB,ATL,17.12,17.12,...,1.85,"$7,400","$7,000",+400,74.6,74.6,74.6,74.6,74.6,74.6


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,...,reception_tds,This Week Salary,Last Week Salary,Salary Differential,wk6 %rostered,wk5 %rostered,wk4 %rostered,wk3 %rostered,wk2 %rostered,wk1 %rostered
0,2024,6,4360569,Jordan Mason,Jordan,Mason,RB,SF,13.85,15.47,...,0.05,"$8,900","$8,700",+200,94.8,94.8,94.8,94.8,94.8,94.8
1,2024,6,3043078,Derrick Henry,Derrick,Henry,RB,BAL,15.55,16.92,...,0.06,"$9,100","$9,200",-100,100.0,100.0,100.0,100.0,100.0,100.0
2,2024,6,3929630,Saquon Barkley,Saquon,Barkley,RB,PHI,15.06,17.93,...,0.15,"$9,200",-,-,100.0,100.0,100.0,100.0,100.0,100.0
3,2024,6,4047365,Josh Jacobs,Josh,Jacobs,RB,GB,11.69,13.90,...,0.10,"$7,600","$7,000",+600,98.9,98.9,98.9,98.9,98.9,98.9
4,2024,6,4241416,Chuba Hubbard,Chuba,Hubbard,RB,CAR,12.41,15.30,...,0.09,"$7,400","$6,800",+600,89.5,89.5,89.5,89.5,89.5,89.5


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,...,reception_tds,This Week Salary,Last Week Salary,Salary Differential,wk6 %rostered,wk5 %rostered,wk4 %rostered,wk3 %rostered,wk2 %rostered,wk1 %rostered
0,2024,6,4258173,Nico Collins,Nico,Collins,WR,HOU,NaN,NaN,...,NaN,"$8,900","$8,800",+100,99.2,99.2,99.2,99.2,99.2,99.2
1,2024,6,4362628,Ja'Marr Chase,Ja'Marr,Chase,WR,CIN,11.75,17.85,...,0.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024,6,4047650,DK Metcalf,DK,Metcalf,WR,SEA,9.96,15.15,...,0.45,"$8,300","$8,200",+100,98.6,98.6,98.6,98.6,98.6,98.6
3,2024,6,4262921,Justin Jefferson,Justin,Jefferson,WR,MIN,0.00,0.00,...,0.00,NaN,NaN,NaN,100.0,100.0,100.0,100.0,100.0,100.0
4,2024,6,4362249,Jayden Reed,Jayden,Reed,WR,GB,10.27,15.36,...,0.42,"$7,700","$7,200",+500,94.3,94.3,94.3,94.3,94.3,94.3
